In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None #for not showing warning message 
pd.options.display.float_format = '{:,.2f}'.format #for setting thhe float format to this notebook

# Reading Master Data

In [9]:
jccycle                   =      pd.read_excel("D:\Analytics\Marketing Report\Journey Cycle\Master Journey Cycle.xlsx")
geodata                   =      pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Geodata.csv")
product_hierarchy         =      pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Product_Hierarchy.csv")
static_category_lookup    =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Static_Category_Lookup.xlsx")
retailer_latest           =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Retailer_Latest.xlsx")
consolidated_graph        =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Consolidated_Graph.xlsx")
distributor_type          =      pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Distributor_Type.xlsx")
active_retailers_PerJC    =      pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Active_Retailers_PerJC.csv",
                                              low_memory=False)

In [123]:
geodata                   =      pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Geodata.csv")

# Reading the dataset from FY 19-20 folder

In [21]:
data = pd.read_excel("D:\Analytics\Marketing Report\FY 19-20\RS CONNECT MIS REPORT JC 07.xlsx",sheet_name = "Sheet1")

In [23]:
backup = data

# Data Cleaning - Imputing Nulls and Feature Selection

In [24]:
data.columns = map(str.lower, data.columns)
data.columns

Index(['statename', 'distcode', 'distname', 'disttype', 'salesmancode',
       'salesmanname', 'routecode', 'routename', 'rtrid', 'rtrcode', 'rtrname',
       'rtrgroupcde', 'rtrclasscde', 'billno', 'invdate', 'brand', 'b.pack',
       'prdcode', 'prdname', 'totqty', 'grossamt', 'taxableamt', 'netamt',
       'basevalue', 'div'],
      dtype='object')

In [25]:
data = data.rename(columns = {"distname" : "cusname",
                              "disttype" : "cusgroname",
                              "rtrgroupcde" : "category"})

### Data Cleaning Function

1. Changing the column names to lower case.    
2. null_cols variable will show which column has the null values. Only taxable ammount can have null values.
3. Imputing the NA values in category as <b>"Kirana"</b>
4. And Imputing the <b>rtrcode</b> and <b>rtrname</b> NA values as <b>"Data not found"</b>
5. Merging the data with <b>Master JC  cycle</b> created and finding the JC value for that data.
6. After Merging the columns used are
                distcode
                rtrcode
                prdcode
                cusname
                cusgroname
                billno
                invdate
                totqty
                basevalue
                div
                Jc
                Week
        
                

        
    

In [26]:
def datacleaning(dataframe):
    data2               = dataframe
    data2.columns       = map(str.lower, data2.columns)
    data2               = data2[["distcode","statename","rtrcode","category","prdcode","cusname","cusgroname",
                                 "billno","invdate","totqty","basevalue","div"]]
    data2["category"]   = data2["category"].fillna(value = "Kirana")
    data2["rtrcode"]    = data2["rtrcode"].fillna(value = "Data not found")
    null_cols           = data2.isnull().sum().to_frame().reset_index()
#     null_cols[null_cols[0] != 0]
    data2               = pd.merge(data2, jccycle, left_on = "invdate", right_on = "Wk_Start_Date")
    data2.drop(columns  = ["Wk_Start_Date"] ,axis = 1,inplace = True)
    print (null_cols)
    print(" \n If the above values are zero proceed to next step")
    return data2

In [27]:
data = datacleaning(data)

         index  0
0     distcode  0
1    statename  0
2      rtrcode  0
3     category  0
4      prdcode  0
5      cusname  0
6   cusgroname  0
7       billno  0
8      invdate  0
9       totqty  0
10   basevalue  0
11         div  0
 
 If the above values are zero proceed to next step


In [125]:
print("")

# Appending Master Data 1 - Geodata

In [124]:
geodata_new      = data[["statename","distcode"]]
geodata_new      = geodata.drop_duplicates()
geodata_check    = pd.merge(geodata_new,geodata["distcode"],on = "distcode",how = "left",indicator = True)
geonew_check     = geodata_check[geodata_check["_merge"] != 'both']
geonew_count     = geonew_check[["statename","distcode"]]

if len(geonew_count) > 0:
    geodata      = geodata.append(geonew_count)
    geodata.to_csv  ("D:\Analytics\Marketing Report\FY 19-20\Master Data\Geodata.csv",index = False)
    print           ("New data apppended to the Master Table - Geodata")
else:
    print           ("There are no new data to append the Master Table Geodata")

print(geonew_count)

There are no new data to append the Master Table Geodata
Empty DataFrame
Columns: [statename, distcode]
Index: []


### Cleaned data is moved to new folder

In [21]:
data.to_csv("D:\Analytics\Marketing Report\FY 19-20\Cleaned data\Cleaned_JC03.csv",index = False)

# Code to find active retailers and All retailers

In [ ]:
data = pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Cleaned data\Cleaned_JC06.csv")
data.shape

In [ ]:
data["Jc"].unique()[0]

In [ ]:
def retailer_classifier(data):
    a = data["Jc"].unique()[0]
    jc_active     = data.groupby(["Jc","rtrcode"]).agg(Active = ("Week","nunique"), 
                                                   Sales = ("basevalue","sum"),
                                                   JC_count = ("rtrcode","count")
                                                  ).reset_index()
    # act_retailers.sort_values(by = "Active", ascending= False)
#   jc_active = pd.merge(retailer_master,act_retailers[["rtrcode","Active","Sales"]], how = "left", on = "rtrcode")
    #From the input cleaned dataset after merging with JC cycle  table we are using the Jc  column as a 
    # dynamic variable for classifying the retailers.
     
#     jc_active['JC' + str(a) + '_Active'] =  np.nan
    jc_active = jc_active.rename(columns = {"JC_count" : 'JC' + str(a) + '_count'})
    jc_active.loc[jc_active['Active'].isin([1,2,3,4]),'JC' + str(a) + '_Active'] = 'All'
    jc_active.loc[jc_active['Active'].isin([4]),'JC' + str(a) + '_Active'] = 'Active'
    jc_active.loc[jc_active['Active'].isnull(),'JC' + str(a) + '_Active'] = 'NULL'
    jc_active['JC' + str(a) + '_Sales'] = jc_active["Sales"] * 100000
    jc_active['JC' + str(a) + '_Throughput'] = jc_active['JC' + str(a) + '_Sales'] / jc_active['JC' + str(a) + '_count']
    jc_active = jc_active[["rtrcode",
                             'JC' + str(a) + '_Active',
                             'JC' + str(a) + '_count',
                             'JC' + str(a) + '_Sales',
                             'JC' + str(a) + '_Throughput']]
    return jc_active

In [ ]:
jc_active = retailer_classifier(data)

In [ ]:
jc_active.head(3)

In [ ]:
active_retailer_perjc = pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Active_Retailers_PerJC.csv")

In [ ]:
newdata = pd.merge(active_retailer_perjc,jc_active,
                  on = "rtrcode",
                  how="left")

In [ ]:
newdata.head(3)

In [ ]:
newdata.to_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Active_Retailers_PerJC.csv",index=False)

# Throughput Graph

In [114]:
active_retailers_perjc = pd.read_csv("D:\Analytics\Marketing Report\FY 19-20\Master Data\Active_Retailers_PerJC.csv")

C:\Users\DELL\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (1,5,9,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [131]:
a = 6
tempdata = active_retailers_perjc[["JC" + str(a) + "_Active",
                                   "JC" + str(a) + "_count",
                                   "JC" + str(a) + "_Sales",
                                   "JC" + str(a) + "_Throughput"]]

temp_groupby = tempdata.groupby(["JC" + str(a) + "_Active"]).agg(Count = ("JC" + str(a) + "_count","sum"),
                                                                Sales = ("JC" + str(a) + "_Sales","sum"),
                                                                Throughput = ("JC" + str(a) + "_Throughput","sum"))

temp_graph = pd.DataFrame(temp_groupby).reset_index()
temp_graph["JC"] = a
temp_graph = temp_graph.rename(columns = {"JC" + str(a) + "_Active":"Active"})
graph = graph.append(temp_graph)
# graph = temp_graph
graph.head(2)

,Active,Count,Sales,Throughput,JC
0,Active,437.00,"98,100.33","3,932.13",1
1,All,"264,144.00","403,977,135.32","80,719,352.91",1
0,Active,"65,068.00","135,186,255.77","4,727,452.01",2
1,All,"454,109.00","540,807,806.95","113,683,898.50",2
0,Active,"75,353.00","196,319,599.26","6,413,080.06",3
1,All,"553,954.00","951,911,131.11","186,738,101.40",3
0,Active,"101,404.00","252,307,100.73","7,762,265.04",4
1,All,"714,488.00","713,025,938.65","138,396,306.59",4
0,Active,"101,408.00","211,941,874.36","7,188,252.67",5
1,All,"835,732.00","612,106,418.55","125,513,838.76",5


In [132]:
graph.to_excel("D:\Analytics\Marketing Report\FY 19-20\Master Data\Consolidated_Graph.xlsx",index = False)